# Testing The API

## Imports

In [2]:
from typing import List
from pydantic import BaseModel
from enum import Enum
from fastapi import FastAPI
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Neo4jVector
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA  # Q&A retrieval system.
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter
import uvicorn
import os
import langchain
from datetime import datetime
from pydantic import BaseModel


/Users/quintenrosseel/Development/llmfaq/api/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Model

In [1]:
## DATA MODEL =============================================
class FeedbackEnum(str, Enum):
    """ 
        A Pydantic model representing feedback. 
    """
    like = 1
    neutral = 0
    dislike = -1

class Prompt(BaseModel):
    """
    A Pydantic model representing a LLM prompt
    """
    prompt: str

class Question(BaseModel):
    """
    A Pydantic model representing a search question. 
    """
    search_string: str

class ChunkMetadata(BaseModel):
    """
    A Pydantic model representing metadata.
    """
    chunk_size: int
    qa_embedding_model: str
    retrieval_embedding_model: str
    chunk_order: int
    chunk_overlap: int
    chunk_id: int

class Document(BaseModel):
    """
    A Pydantic model representing a document.
    Wrapper around langchains langchain.schema.document.Document.
    """
    page_content: str
    metadata: ChunkMetadata

class Answer(BaseModel):
    """
    A Pydantic model representing an answer to a question.
    """
    context: List[Document]
    llm_prompt: str
    llm_answer: str
    language: str
    score: FeedbackEnum


NameError: name 'Enum' is not defined

## Constants

In [19]:
BASE_PROMPT_TEMPLATE = """
ANSWER THE QUESTION BASED ONLY ON THE FOLLOWING CONTEXT:
{context}

QUESTION: 
{question}

ANSWER IN THE FOLLOWING LANGUAGE: 
{language}

CLEARLY STATE IF THE ANSWER CANNOT BE FOUND IN THE CONTEXT ABOVE.
IF THE ANSWER CAN BE FOUND, REFERENCE THE CONTEXT. 
"""
# Define the configuration
RETRIEVER_SEARCH_CONFIG = {
    # "similarity" (default), "mmr", or "similarity_score_threshold".
    'search_type': 'similarity', 
    'search_kwargs': {
        # Amount of documents to return (default: 4).
        'k': 5, 
        # Amount of documents to pass to the MMR algorithm 
        # # (default: 20).
        'fetch_k': 50, 
        # Minimum relevance threshold for similarity_score_threshold.
        'score_threshold': 0, 
        # Diversity of results returned by MMR; 
        # # 1 for minimum diversity and 0 for maximum (default: 0.5).
        'lambda_mult': 0.25, 
        # Filter by document metadata.
        'filter': {'chunk_size': 500}
    }
}


## Global Variables

In [13]:
#### GLOBAL VARIABLES =================
langchain.verbose = False
langchain.debug = False
load_dotenv() 

# Instructor Embeddings

model_kwargs = {
    'device': 'cpu'
}

encode_kwargs = {
    # 'normalize_embeddings': True,
    'show_progress_bar': False
}

instructor_model = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-xl", 
    cache_folder='models/hkunlp_instructor-xl',
    embed_instruction="Represent the Medical question for retrieving supporting paragraphs: ",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

robbert_model = HuggingFaceEmbeddings(
    model_name="jegorkitskerkin/robbert-v2-dutch-base-mqa-finetuned", 
    cache_folder='models/robbert-v2-dutch-base-mqa-finetuned',
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


# Neo4j driver
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USER"), 
        os.getenv("NEO4J_PASSWORD")
    )
)


load INSTRUCTOR_Transformer
max_seq_length  512


## Helper Functions

In [22]:
from langchain.docstore.document import Document

def docs_to_str(
        l: List[langchain.schema.document.Document], 
        include_metadata: bool = True,
        skip_meta_keys: List[str] = []) -> str:
    page_content: str = ""
    for i, d in enumerate(l): 
        # Page Content
        page_content += (
            # (25 * "=") + 
            (f"\n## Document {i+1} ") + 
            # (25 * "=") + 
            '\n'
        )

        if include_metadata:
            page_content += (
                # (25 * "=" ) + 
                (f"### Metadata ") + 
                # (5 * "=" ) + 
                '\n'
            )
            # Metadata 
            for k, v in d.metadata.items():
                if not k in skip_meta_keys:
                    page_content += (f"- {k}: {v} \n")
        page_content += (
            # (25 * "=" ) + 
            (f"### Inhoud ") + 
            # (5 * "=" ) + 
            '\n'
        )
        page_content += d.page_content
        page_content += '\n'
    return page_content


def chunk_paths_to_docs(chunk_paths: List[object]) -> List[Document]:
    """
        For now, this matches chunk_paths of type (Chunk)-rel-(WebPage)-()
    """
    # This returns paths, that we can turn into LangChain documents somehow. 
    path_docs: List[Document] = []

    # One result for every chunk (see above)
    for p in chunk_paths:
        chunk_path_str = '' 
        chunk_node = p['rel'][0] 
        chunk_text = chunk_node.get('text')
        
        # Build up metadata of Document object manually
        doc_meta = {
            'chunk_size': chunk_node.get('chunk_size'),
            'qa_embedding_model': chunk_node.get('qa_embedding_model'),
            'retrieval_embedding_model': chunk_node.get('retrieval_embedding_model'),
            'chunk_order': chunk_node.get('chunk_order'),
            'chunk_overlap': chunk_node.get('chunk_overlap'),
            'chunk_id': chunk_node.get('chunk_id'),
        }
        # Traverse path for metadata
        for i, o in enumerate(p['rel']): 
            # Create path representation
            if type(o) == dict:
                chunk_path_str += f'(Node)'
            elif type(o) == str:
                chunk_path_str += f'<-{o}-'
            
            # WebPage node data
            if i == 2:
                doc_meta['webpage_scrape_dt'] = o.get('scrape_dt')
                doc_meta['webpage_url'] = o.get('url')
                doc_meta['webpage_title'] = o.get('title')
            # Catalog node data
            elif i == 4: 
                doc_meta['catalog_url'] = o.get('url')
        
        # Add path structure as metadata
        doc_meta['path_context'] = chunk_path_str   
        
        # Extract metadata from traversal 
        path_docs.append(
            Document(
                page_content=chunk_text, 
                metadata=doc_meta
            )
        )
    return path_docs


def get_neo4j_node_paths( 
        db: Neo4jVector, 
        question: str, 
        embedding_model: object,
        embedding_index: str, 
        limit: int = 5,) -> List[langchain.schema.document.Document]:
    vec_chunk_paths = db.query(
        f"""
            CALL db.index.vector.queryNodes(
                '{embedding_index}', 
                {limit},
                {embedding_model.embed_query(question)}
            ) 
            YIELD node, score
            WITH node, score
            ORDER BY score DESCENDING
            MATCH rel=(node:Chunk)<-[:HAS_CHUNK]-(:WebPage)<-[:HAS_WEBPAGE]-(:Catalog)
            RETURN DISTINCT rel
        """
    )
    return vec_chunk_paths

def question_to_context(
        question: str, 
        graph: Neo4jVector,
        limit=5,
        embedding_model=robbert_model,
        embedding_index="vi_chunk_qa_embedding_cosine",
        to_str: bool = False
        ) -> any:
    neo4j_paths = get_neo4j_node_paths(
        db=graph, 
        question=question, 
        embedding_model=embedding_model,
        embedding_index=embedding_index, 
        limit=limit,
    )

    neo4j_docs: List[Document] = chunk_paths_to_docs(neo4j_paths)
    if to_str:
        return docs_to_str(neo4j_docs)
    else:
        return neo4j_docs


## Test the API

In [17]:
import textwrap
from langchain.chains import LLMChain

BASE_PROMPT_TEMPLATE_NL = """
Je bent een ziekenhuismedewerker en je moet op een vraag van een patiënt antwoorden op basis van paragrafen op de website van het ziekenhuis.
Beantwoord de vraag op basis van de volgende context (paragrafen op de website):
```
{context}
```

De vraag is: 
```
{question}
```

Instructies bij het antwoorden: 
- Algemeen:
    - Antwoord altijd in het {language}
    - Geef duidelijk aan als het antwoord niet gevonden kan worden in de context (paragrafen op de website). 
- Wanneer je niets vindt in de context, vermeld dan:
    - Een lijst van de meest gelijkende paragrafen op de website. LIJST DE MEEST GELIJKE CONTEXT OP MET KOMMA'S
    - De weburl naar de paragraaf op de website. (indien gegeven)
- Wanneer je wel iets vindt in de context, vermeld dan:
    - Een Markdown lijst met de relevante context, gescheiden door komma's.
    - De weburl (bron) van de context. (indien gegeven)
"""

# QUESTION =============================================
question = "Wat moet ik doen bij nierstenen?"

## RETRIEVE DOCUMENTS =================

# Graph from existing graph
neo4j_qa_graph = Neo4jVector.from_existing_index(
    embedding=robbert_model,
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="vi_chunk_qa_embedding_cosine",
    keyword_index_name="fts_Chunk_text",
    search_type="hybrid",
)

neo4j_retrieval_graph = Neo4jVector.from_existing_index(
    embedding=instructor_model,
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="vi_chunk_retrieval_embedding_cosine",
    keyword_index_name="fts_Chunk_text",
    search_type="hybrid",
)


llm_prompt = PromptTemplate(
    input_variables=["context", "question", "language"],
    template=BASE_PROMPT_TEMPLATE_NL,
)


llm = ChatOpenAI(
    temperature=0
)
llm_chain = LLMChain(
    prompt=llm_prompt, 
    llm=llm,
    verbose=False
)

# Evaluate QA Output
- Keeping Prompt and LLM Stable

In [26]:
    neo4j_qa_retriever.get_relevant_documents(
        question
    )

[Document(page_content='## Wanneer kan ik me laten testen?\n\n\nEen afname in het lab kan kan gebeuren op onderstaande momenten:\n\n\n* maandag - vrijdag: van 10u tot 12u en van 14u tot 18u\n* zaterdag: van 10u tot 12u\n\nJe schrijft hiervoor eerst in aan de kiosken aan het onthaal.\n\n\n\n## Wanneer en hoe is het resultaat beschikbaar?\n\nBinnen de 24u\\* na afname is het resultaat van je PCR-test beschikbaar:', metadata={'chunk_size': 500, 'retrieval_embedding_model': 'hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval', 'retrieval_embedding': [0.05550762638449669, -0.008720907382667065, -0.009237530641257763, -0.05045356601476669, -0.008795774541795254, -0.010577857494354248, -0.046653859317302704, 0.018743939697742462, -0.01688176579773426, -0.03587788715958595, -0.015616183169186115, -0.0026292793918401003, -0.029646053910255432, -0.1191941574215889, -0.011612179689109325, 0.005081630311906338, 0.0011925884755328298, -0.03757712244987488, 0.045944

In [27]:

question = "Mag mijn baby na 12u naar huis na geboorte?"

# Langchain retrievers  =============
neo4j_qa_retriever = neo4j_qa_graph.as_retriever(**RETRIEVER_SEARCH_CONFIG)

qa_docs: List[langchain.schema.document.Document] = (
    neo4j_qa_retriever.get_relevant_documents(
        question
    )
)

In [28]:
from IPython.display import display_markdown

qa_llm_answer = llm_chain.invoke(
    input={
        "question": question,
        "context": docs_to_str(
            qa_docs,
            include_metadata=False),
        "language": "Nederlands"
    },
    return_only_outputs=False,
    include_run_info=True
)


display_markdown(
    "# Base LLM context", 
    BASE_PROMPT_TEMPLATE_NL.format_map(qa_llm_answer), 
    "# Base LLM Antwoord", 
    qa_llm_answer['text'], 
    raw=True
)

# Base LLM context


Je bent een ziekenhuismedewerker en je moet op een vraag van een patiënt antwoorden op basis van paragrafen op de website van het ziekenhuis.
Beantwoord de vraag op basis van de volgende context (paragrafen op de website):
```

## Document 1 
### Inhoud 
## Ontslag


Nadat de interventionele radioloog op de kamer langs is geweest, mag je naar huis. Je dient daarbij rekening te houden met het volgende: 

• Je wordt naar huis gebracht, je mag niet zelf met de wagen rijden.


• Hou het drukverband ter plaatse tot 12 uur na je ontslag uit het ziekenhuis. De prikplaats mag daarna met een kleefpleister afgedekt worden.

## Document 2 
### Inhoud 
Na het verwijderen van de naald(en) wordt een doucheverbandje aangelegd. Je hoeft geen verdere wondzorg te voorzien. 



## Na de behandeling


Na de behandeling mag je onmiddellijk naar huis. Indien nodig kan een bijkomende afspraak gemaakt worden aan het secretariaat.

## Document 3 
### Inhoud 
Het ontslag gebeurt na goedkeuring van de arts. Je mag niet zelf naar huis rijden, iemand moet je ophalen. Bij het ontslag krijg je een datum mee voor een controle.

## Document 4 
### Inhoud 
## Na de behandeling


Na de behandeling kun je gewoon naar huis. Dit moet je weten:

## Document 5 
### Inhoud 
## Na het onderzoek


Na het onderzoek mag je meteen naar huis. Gebruik van wagen, fiets of openbaar vervoer is geen probleem.


De gastro-enteroloog beoordeelt de metingen en stelt een verslag op. Je krijgt de uitslag van het onderzoek via je behandelend arts.

```

De vraag is: 
```
Mag mijn baby na 12u naar huis na geboorte?
```

Instructies bij het antwoorden: 
- Algemeen:
    - Antwoord altijd in het Nederlands
    - Geef duidelijk aan als het antwoord niet gevonden kan worden in de context (paragrafen op de website). 
- Wanneer je niets vindt in de context, vermeld dan:
    - Een lijst van de meest gelijkende paragrafen op de website. LIJST DE MEEST GELIJKE CONTEXT OP MET KOMMA'S
    - De weburl naar de paragraaf op de website. (indien gegeven)
- Wanneer je wel iets vindt in de context, vermeld dan:
    - Een Markdown lijst met de relevante context, gescheiden door komma's.
    - De weburl (bron) van de context. (indien gegeven)


# Base LLM Antwoord

Het antwoord op de vraag "Mag mijn baby na 12u naar huis na geboorte?" kan niet gevonden worden in de context (paragrafen op de website). De meest gelijkende paragrafen op de website zijn: "Ontslag" (Document 1), "Na de behandeling" (Document 2), "Het ontslag gebeurt na goedkeuring van de arts" (Document 3), "Na de behandeling" (Document 4), "Na het onderzoek" (Document 5). 

Bron: N/A

In [75]:
# Manual approach (circumvent langchain retrieval process) =============
enriched_qa_docs: List[Document] = question_to_context(
    question=question, 
    graph=neo4j_qa_graph,
    limit=5,
    embedding_model=robbert_model,
    embedding_index="vi_chunk_qa_embedding_cosine",
    to_str=False
) 


In [76]:
enriched_qa_docs[0]

Document(page_content='In de preventie van nieuwe nierstenen is het soms aangewezen om een [metabool nazicht](https://azsintlucas-be.vm01.glue.be/onderzoek-en-behandelingen/metabool-bilan) te laten doen.\n\n\n\n# Geneesmiddelen\n\n\nWanneer men een duidelijk aanwijsbare oorzaak voor de nierstenen vindt, stelt de arts een behandeling met geneesmiddelen in. Sommige van die behandelingen moet je lang volhouden. Enkele voorbeelden:', metadata={'chunk_size': 500, 'qa_embedding_model': 'egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned', 'retrieval_embedding_model': 'hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval', 'chunk_order': 316, 'chunk_overlap': 60, 'chunk_id': 316, 'webpage_scrape_dt': '12/11/2023 15:27:12', 'webpage_url': 'https://www.azstlucas.be/onderzoek-en-behandelingen/behandeling-van-nierstenen', 'webpage_title': 'Behandeling van nierstenen', 'catalog_url': 'https://www.azstlucas.be/onderzoek-en-behandelingen', 'path_context': '(Node)<-HAS

In [78]:
enriched_qa_llm_answer = llm_chain.invoke(
    input={
        "question": question,
        "context": docs_to_str(
            enriched_qa_docs,
            include_metadata=True, 
            skip_meta_keys=[
                'chunk_id', 
                'chunk_order', 
                'chunk_overlap', 
                'chunk_size', 
                'qa_embedding_model', 
                'retrieval_embedding_model'
            ]
        ), 
        "language": "Nederlands"
    },
    return_only_outputs=False,
    include_run_info=True
)

display_markdown(
    "# Enriched LLM Context", 
    # Use input variables from chain in map
    BASE_PROMPT_TEMPLATE_NL.format_map(enriched_qa_llm_answer), 
    "# Enriched LLM Antwoord", 
    enriched_qa_llm_answer['text'], 
    raw=True
)

# Enriched LLM Context


Je bent een ziekenhuismedewerker en je moet op een vraag van een patiënt antwoorden op basis van paragrafen op de website van het ziekenhuis.
Beantwoord de vraag op basis van de volgende context (paragrafen op de website):
```

## Document 1 
### Metadata 
- webpage_scrape_dt: 12/11/2023 15:27:12 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/behandeling-van-nierstenen 
- webpage_title: Behandeling van nierstenen 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
In de preventie van nieuwe nierstenen is het soms aangewezen om een [metabool nazicht](https://azsintlucas-be.vm01.glue.be/onderzoek-en-behandelingen/metabool-bilan) te laten doen.



# Geneesmiddelen


Wanneer men een duidelijk aanwijsbare oorzaak voor de nierstenen vindt, stelt de arts een behandeling met geneesmiddelen in. Sommige van die behandelingen moet je lang volhouden. Enkele voorbeelden:

## Document 2 
### Metadata 
- webpage_scrape_dt: 12/11/2023 15:27:12 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/behandeling-van-nierstenen 
- webpage_title: Behandeling van nierstenen 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
Een niersteencrisis wordt in de eerste plaats behandeld met medicatie. Soms schrijft de arts pijnstillende medicatie voor, gecombineerd met spierontspannende middelen. Hierdoor worden de urineleiders wat breder zodat de steen er gemakkelijker doorheen kan. Het is belangrijk dat je voldoende drinkt, zodat de stenen weggespoeld kunnen worden. In de meerderheid van de gevallen volstaat het geduld te hebben en zal de steen uiteindelijk vanzelf met de urine worden geloosd. Tot het zover is, moet je

## Document 3 
### Metadata 
- webpage_scrape_dt: 12/11/2023 15:31:06 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/ureter-reimplantatie 
- webpage_title: Ureter-reïmplantatie 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
Zelf als het niet pijnlijk is kan het aangewezen zijn om deze aandoening te behandelen aangezien de werking van die nier geleidelijk aan achteruit gaat. De kans op een infectie of steenvorming in die nier is ook groter.

## Document 4 
### Metadata 
- webpage_scrape_dt: 12/11/2023 15:27:12 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/behandeling-van-nierstenen 
- webpage_title: Behandeling van nierstenen 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
* [Ureterorenoscopie](https://azsintlucas-be.vm01.glue.be/onderzoek-en-behandelingen/ureterorenoscopie)
* [Steenverbrijzeling (ESWL)](https://azsintlucas-be.vm01.glue.be/onderzoek-en-behandelingen/steenverbrijzeling-eswl)
* [Percutane niersteenbehandeling](https://azsintlucas-be.vm01.glue.be/onderzoek-en-behandelingen/percutane-niersteenbehandeling)

## Document 5 
### Metadata 
- webpage_scrape_dt: 12/11/2023 15:29:20 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/metabool-bilan 
- webpage_title: Metabool bilan 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
Er zijn verschillende types nierstenen (zes om precies te zijn), met elk hun eigen oorzaak en ontstaansmechanisme. Als je het type steen en de risicofactoren kent, kun je behandeling daarop afstemmen en is advies op maat mogelijk. 


Heel concreet geeft het metabool bilan een antwoord op de volgende vragen:


* Waarom maak ik nierstenen aan?
* Welke maatregelen zijn nodig om mijn kans op herval te verminderen.



## Voor wie?

```

De vraag is: 
```
Wat moet ik doen bij nierstenen?
```

Instructies bij het antwoorden: 
- Algemeen:
    - Antwoord altijd in het Nederlands
    - Geef duidelijk aan als het antwoord niet gevonden kan worden in de context (paragrafen op de website). 
- Wanneer je niets vindt in de context, vermeld dan:
    - Een lijst van de meest gelijkende paragrafen op de website. LIJST DE MEEST GELIJKE CONTEXT OP MET KOMMA'S
    - De weburl naar de paragraaf op de website. (indien gegeven)
- Wanneer je wel iets vindt in de context, vermeld dan:
    - Een Markdown lijst met de relevante context, gescheiden door komma's.
    - De weburl (bron) van de context. (indien gegeven)


# Enriched LLM Antwoord

Het antwoord op de vraag "Wat moet ik doen bij nierstenen?" kan niet gevonden worden in de context. Hieronder vind je een lijst van de meest gelijkende paragrafen op de website:

1. "Geneesmiddelen" - https://www.azstlucas.be/onderzoek-en-behandelingen/behandeling-van-nierstenen
2. "Een niersteencrisis wordt in de eerste plaats behandeld met medicatie." - https://www.azstlucas.be/onderzoek-en-behandelingen/behandeling-van-nierstenen
3. "Zelf als het niet pijnlijk is kan het aangewezen zijn om deze aandoening te behandelen." - https://www.azstlucas.be/onderzoek-en-behandelingen/ureter-reimplantatie
4. "Ureterorenoscopie, Steenverbrijzeling (ESWL), Percutane niersteenbehandeling" - https://www.azstlucas.be/onderzoek-en-behandelingen/behandeling-van-nierstenen
5. "Er zijn verschillende types nierstenen (zes om precies te zijn), met elk hun eigen oorzaak en ontstaansmechanisme." - https://www.azstlucas.be/onderzoek-en-behandelingen/metabool-bilan

Helaas kan ik geen specifiek antwoord geven op de vraag. Het is raadzaam om contact op te nemen met een arts of medisch professional voor advies over wat te doen bij nierstenen.

# Retrieval Model

In [30]:
neo4j_retrieval_retriever = neo4j_retrieval_graph.as_retriever(**RETRIEVER_SEARCH_CONFIG)

retrieval_docs: List[langchain.schema.document.Document] = (
    neo4j_retrieval_retriever.get_relevant_documents(
        question
    )
)

enriched_retrieval_docs: str = question_to_context(
    question=question, 
    graph=neo4j_retrieval_graph,
    limit=5,
    embedding_model=instructor_model,
    embedding_index="vi_chunk_retrieval_embedding_cosine"
) 

In [37]:
retrieval_llm_answer = llm_chain.invoke(
    input={
        "question": question,
        "context": docs_to_str(
            retrieval_docs, 
            skip_meta_keys=[
                'qa_embedding', 
                'retrieval_embedding'
            ]), 
        "language": "Nederlands"
    },
    return_only_outputs=False,
    include_run_info=True
)

display_markdown(
    "# Enriched LLM Context", 
    # Use input variables from chain in map
    BASE_PROMPT_TEMPLATE_NL.format_map(retrieval_llm_answer), 
    "# Enriched LLM Antwoord", 
    retrieval_llm_answer['text'], 
    raw=True
)

# Enriched LLM Context


Je bent een ziekenhuismedewerker en je moet op een vraag van een patiënt antwoorden op basis van paragrafen op de website van het ziekenhuis.
Beantwoord de vraag op basis van de volgende context (paragrafen op de website):
```

## Document 1 
### Metadata 
- chunk_size: 500 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 95 
- chunk_overlap: 60 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- chunk_id: 95 
### Inhoud 
## Ontslag


Nadat de interventionele radioloog op de kamer langs is geweest, mag je naar huis. Je dient daarbij rekening te houden met het volgende: 

• Je wordt naar huis gebracht, je mag niet zelf met de wagen rijden.


• Hou het drukverband ter plaatse tot 12 uur na je ontslag uit het ziekenhuis. De prikplaats mag daarna met een kleefpleister afgedekt worden.

## Document 2 
### Metadata 
- chunk_size: 500 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 2322 
- chunk_overlap: 60 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- chunk_id: 2322 
### Inhoud 
Na het verwijderen van de naald(en) wordt een doucheverbandje aangelegd. Je hoeft geen verdere wondzorg te voorzien. 



## Na de behandeling


Na de behandeling mag je onmiddellijk naar huis. Indien nodig kan een bijkomende afspraak gemaakt worden aan het secretariaat.

## Document 3 
### Metadata 
- chunk_size: 500 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 1006 
- chunk_overlap: 60 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- chunk_id: 1006 
### Inhoud 
Het ontslag gebeurt na goedkeuring van de arts. Je mag niet zelf naar huis rijden, iemand moet je ophalen. Bij het ontslag krijg je een datum mee voor een controle.

## Document 4 
### Metadata 
- chunk_size: 500 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 3790 
- chunk_overlap: 60 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- chunk_id: 3790 
### Inhoud 
## Na de behandeling


Na de behandeling kun je gewoon naar huis. Dit moet je weten:

## Document 5 
### Metadata 
- chunk_size: 500 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 3719 
- chunk_overlap: 60 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- chunk_id: 3719 
### Inhoud 
## Na het onderzoek


Na het onderzoek mag je meteen naar huis. Gebruik van wagen, fiets of openbaar vervoer is geen probleem.


De gastro-enteroloog beoordeelt de metingen en stelt een verslag op. Je krijgt de uitslag van het onderzoek via je behandelend arts.

```

De vraag is: 
```
Mag mijn baby na 12u naar huis na geboorte?
```

Instructies bij het antwoorden: 
- Algemeen:
    - Antwoord altijd in het Nederlands
    - Geef duidelijk aan als het antwoord niet gevonden kan worden in de context (paragrafen op de website). 
- Wanneer je niets vindt in de context, vermeld dan:
    - Een lijst van de meest gelijkende paragrafen op de website. LIJST DE MEEST GELIJKE CONTEXT OP MET KOMMA'S
    - De weburl naar de paragraaf op de website. (indien gegeven)
- Wanneer je wel iets vindt in de context, vermeld dan:
    - Een Markdown lijst met de relevante context, gescheiden door komma's.
    - De weburl (bron) van de context. (indien gegeven)


# Enriched LLM Antwoord

Het antwoord op de vraag "Mag mijn baby na 12u naar huis na geboorte?" kan niet worden gevonden in de gegeven context (paragrafen op de website). Hieronder volgt een lijst van de meest gelijkende paragrafen op de website:

1. Ontslag: Nadat de interventionele radioloog op de kamer langs is geweest, mag je naar huis. Je dient daarbij rekening te houden met bepaalde instructies. (Bron: [Website](geen weburl beschikbaar))

2. Na de behandeling: Na de behandeling mag je onmiddellijk naar huis. Indien nodig kan een bijkomende afspraak gemaakt worden aan het secretariaat. (Bron: [Website](geen weburl beschikbaar))

3. Het ontslag gebeurt na goedkeuring van de arts. Je mag niet zelf naar huis rijden, iemand moet je ophalen. Bij het ontslag krijg je een datum mee voor een controle. (Bron: [Website](geen weburl beschikbaar))

4. Na de behandeling: Na de behandeling kun je gewoon naar huis. Er zijn enkele dingen die je moet weten. (Bron: [Website](geen weburl beschikbaar))

5. Na het onderzoek: Na het onderzoek mag je meteen naar huis. Gebruik van wagen, fiets of openbaar vervoer is geen probleem. De gastro-enteroloog beoordeelt de metingen en stelt een verslag op. Je krijgt de uitslag van het onderzoek via je behandelend arts. (Bron: [Website](geen weburl beschikbaar))

In [39]:
enriched_retrieval_llm_answer = llm_chain.invoke(
    input={
        "question": question,
        "context": docs_to_str(
            enriched_retrieval_docs, 
            skip_meta_keys=[
                'qa_embedding', 
                'retrieval_embedding'
            ]),
        "language": "Nederlands"
    },
    return_only_outputs=False,
    include_run_info=True
)


display_markdown(
    "# Enriched Retrieval LLM Context", 
    # Use input variables from chain in map
    BASE_PROMPT_TEMPLATE_NL.format_map(enriched_retrieval_llm_answer), 
    "# Enriched Retrieval LLM Antwoord", 
    retrieval_llm_answer['text'], 
    raw=True
)

# Enriched Retrieval LLM Context


Je bent een ziekenhuismedewerker en je moet op een vraag van een patiënt antwoorden op basis van paragrafen op de website van het ziekenhuis.
Beantwoord de vraag op basis van de volgende context (paragrafen op de website):
```

## Document 1 
### Metadata 
- chunk_size: 500 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 0 
- chunk_overlap: 60 
- chunk_id: 0 
- webpage_scrape_dt: 12/11/2023 15:26:50 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/24-uur-observatie-van-de-pasgeborene 
- webpage_title: 24 uur observatie van de pasgeborene 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
## Waarom willen de kinderartsen dat je baby minstens 24 uur op de dienst verblijft?

Gelukkig worden de meeste baby's helemaal gezond geboren na een normale zwangerschap. Toch is het belangrijk om een pasgeborene de eerste 24 levensuren bij ons in het ziekenhuis te observeren. Er ontstaan soms problemen die kort na de geboorte nog niet duidelijk zijn.



## Mogelijke afwijkingen


### Aangeboren hartafwijking

## Document 2 
### Metadata 
- chunk_size: 500 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 345 
- chunk_overlap: 60 
- chunk_id: 345 
- webpage_scrape_dt: 12/11/2023 15:27:14 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/besnijdenis-bij-het-kind-circumcisie 
- webpage_title: Besnijdenis bij het kind (circumcisie) 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
+ Flesvoeding bij baby’s voor na de ingreep
* Het is handig als je reeds vooraf pijnstilling haalt in de apotheek, paracetamol en/of ibuprofen, of volg hierbij het voorschrift van je arts.
* Het ziekenhuis is vrij toegankelijk, laat daarom waardevolle bezittingen en grote geldbedragen thuis of laat ze niet onbeheerd achter. Het ziekenhuis kan hiervoor niet aansprakelijk gesteld worden.

## Document 3 
### Metadata 
- chunk_size: 500 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 1807 
- chunk_overlap: 60 
- chunk_id: 1807 
- webpage_scrape_dt: 12/11/2023 15:28:40 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/hydrocoele-bij-het-kind 
- webpage_title: Hydrocoele bij het kind 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
infuus heeft, verwijderen we dat pas als je kind voldoende gedronken en gegeten heeft, als het geplast heeft (wat noodzakelijk is voordat uw kind het ziekenhuis mag verlaten) en geen pijnstilling meer krijgt via het infuus. Als je kind voldoende hersteld is van de ingreep, mag het nog wat spelen in de speelkamer. Na toestemming en advies van de arts mag je kind naar huis. Verzekeringspapieren, briefje sociaal verlof, attesten voor school en sport e.d. moeten door de arts worden ingevuld.

## Document 4 
### Metadata 
- chunk_size: 500 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 2618 
- chunk_overlap: 60 
- chunk_id: 2618 
- webpage_scrape_dt: 12/11/2023 15:29:36 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/niet-ingedaalde-teelballen 
- webpage_title: Niet-ingedaalde teelballen 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
infuus heeft, verwijderen we dat pas als je kind voldoende gedronken en gegeten heeft, als het geplast heeft (wat noodzakelijk is voordat uw kind het ziekenhuis mag verlaten) en geen pijnstilling meer krijgt via het infuus. Als je kind voldoende hersteld is van de ingreep, mag het nog wat spelen in de speelkamer. Na toestemming en advies van de arts mag je kind naar huis. Verzekeringspapieren, briefje sociaal verlof, attesten voor school en sport e.d. moeten door de arts worden ingevuld.

## Document 5 
### Metadata 
- chunk_size: 500 
- qa_embedding_model: egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned 
- retrieval_embedding_model: hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval 
- chunk_order: 4 
- chunk_overlap: 60 
- chunk_id: 4 
- webpage_scrape_dt: 12/11/2023 15:26:50 
- webpage_url: https://www.azstlucas.be/onderzoek-en-behandelingen/24-uur-observatie-van-de-pasgeborene 
- webpage_title: 24 uur observatie van de pasgeborene 
- catalog_url: https://www.azstlucas.be/onderzoek-en-behandelingen 
- path_context: (Node)<-HAS_CHUNK-(Node)<-HAS_WEBPAGE-(Node) 
### Inhoud 
De kinderarts onderzoekt je baby in normale omstandigheden binnen de eerste 24 uur. Liefst volgt er een tweede onderzoek in de 24 tot 48 uur nadien. Het onderzoek kort na de geboorte kan een normale uitkomst hebben terwijl veel symptomen en afwijkingen pas duidelijker worden in de volgende dagen.



## Preventieve maatregelen

We nemen enkele voorzorgen om eventuele afwijkingen op tijd op te merken:

```

De vraag is: 
```
Mag mijn baby na 12u naar huis na geboorte?
```

Instructies bij het antwoorden: 
- Algemeen:
    - Antwoord altijd in het Nederlands
    - Geef duidelijk aan als het antwoord niet gevonden kan worden in de context (paragrafen op de website). 
- Wanneer je niets vindt in de context, vermeld dan:
    - Een lijst van de meest gelijkende paragrafen op de website. LIJST DE MEEST GELIJKE CONTEXT OP MET KOMMA'S
    - De weburl naar de paragraaf op de website. (indien gegeven)
- Wanneer je wel iets vindt in de context, vermeld dan:
    - Een Markdown lijst met de relevante context, gescheiden door komma's.
    - De weburl (bron) van de context. (indien gegeven)


# Enriched Retrieval LLM Antwoord

Het antwoord op de vraag "Mag mijn baby na 12u naar huis na geboorte?" kan niet worden gevonden in de gegeven context (paragrafen op de website). Hieronder volgt een lijst van de meest gelijkende paragrafen op de website:

1. Ontslag: Nadat de interventionele radioloog op de kamer langs is geweest, mag je naar huis. Je dient daarbij rekening te houden met bepaalde instructies. (Bron: [Website](geen weburl beschikbaar))

2. Na de behandeling: Na de behandeling mag je onmiddellijk naar huis. Indien nodig kan een bijkomende afspraak gemaakt worden aan het secretariaat. (Bron: [Website](geen weburl beschikbaar))

3. Het ontslag gebeurt na goedkeuring van de arts. Je mag niet zelf naar huis rijden, iemand moet je ophalen. Bij het ontslag krijg je een datum mee voor een controle. (Bron: [Website](geen weburl beschikbaar))

4. Na de behandeling: Na de behandeling kun je gewoon naar huis. Er zijn enkele dingen die je moet weten. (Bron: [Website](geen weburl beschikbaar))

5. Na het onderzoek: Na het onderzoek mag je meteen naar huis. Gebruik van wagen, fiets of openbaar vervoer is geen probleem. De gastro-enteroloog beoordeelt de metingen en stelt een verslag op. Je krijgt de uitslag van het onderzoek via je behandelend arts. (Bron: [Website](geen weburl beschikbaar))